# run spark 2.4 
# apcahe spark review of creating a dataframe and checking the display

In [1]:
import os
import sys

os.environ["SPARK_HOME"] = "/usr/spark2.4.3"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/local/anaconda/bin/python" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/local/anaconda/bin/python"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

In [2]:
import mlflow

/usr/local/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: MLflow support for Python 3.6 is deprecated and will be dropped in an upcoming release. At that point, existing Python 3.6 workflows that use MLflow will continue to work without modification, but Python 3.6 users will no longer get access to the latest MLflow features and bugfixes. We recommend that you upgrade to Python 3.7 or newer.
  """Entry point for launching an IPython kernel.


In [3]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("appName")
sc = SparkContext(conf=conf)

In [4]:
from pyspark.sql.functions import col , rand
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

In [5]:
## practise ##

In [24]:
## create a dataframe of 1 milllion rows

df=(spark.range(1,1000000)
	.withColumn("id",(col("id") /1000).cast("integer"))
	.withColumn("v",rand(seed=1)))

In [25]:
df.show()

+---+--------------------+
| id|                   v|
+---+--------------------+
|  0| 0.13385709732307427|
|  0|  0.5897562959687032|
|  0| 0.01540012100242305|
|  0| 0.22569943461197162|
|  0|  0.9207602095112212|
|  0|  0.6222816020094926|
|  0|  0.1029837279488438|
|  0|  0.6678762139023474|
|  0| 0.06748208566157787|
|  0|  0.5215181769983375|
|  0|  0.8895645473999635|
|  0|   0.321509746474296|
|  0| 0.28823975483307396|
|  0| 0.45999786693699807|
|  0| 0.17320175842535657|
|  0|1.152006730106292E-4|
|  0|  0.7500451351287378|
|  0|   0.545952636289781|
|  0|  0.8441648098982081|
|  0|   0.315473698725972|
+---+--------------------+
only showing top 20 rows



In [27]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- v: double (nullable = false)



In [28]:
df.select("v").show()

+--------------------+
|                   v|
+--------------------+
| 0.13385709732307427|
|  0.5897562959687032|
| 0.01540012100242305|
| 0.22569943461197162|
|  0.9207602095112212|
|  0.6222816020094926|
|  0.1029837279488438|
|  0.6678762139023474|
| 0.06748208566157787|
|  0.5215181769983375|
|  0.8895645473999635|
|   0.321509746474296|
| 0.28823975483307396|
| 0.45999786693699807|
| 0.17320175842535657|
|1.152006730106292E-4|
|  0.7500451351287378|
|   0.545952636289781|
|  0.8441648098982081|
|   0.315473698725972|
+--------------------+
only showing top 20 rows



In [29]:
display(df)

DataFrame[id: int, v: double]

In [30]:
##Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("df_temp")

In [31]:
df.show()

+---+--------------------+
| id|                   v|
+---+--------------------+
|  0| 0.13385709732307427|
|  0|  0.5897562959687032|
|  0| 0.01540012100242305|
|  0| 0.22569943461197162|
|  0|  0.9207602095112212|
|  0|  0.6222816020094926|
|  0|  0.1029837279488438|
|  0|  0.6678762139023474|
|  0| 0.06748208566157787|
|  0|  0.5215181769983375|
|  0|  0.8895645473999635|
|  0|   0.321509746474296|
|  0| 0.28823975483307396|
|  0| 0.45999786693699807|
|  0| 0.17320175842535657|
|  0|1.152006730106292E-4|
|  0|  0.7500451351287378|
|  0|   0.545952636289781|
|  0|  0.8441648098982081|
|  0|   0.315473698725972|
+---+--------------------+
only showing top 20 rows



## use sql to query the dataframe 

In [32]:

spark.sql("select *  from df_temp limit 5").show()

+---+-------------------+
| id|                  v|
+---+-------------------+
|  0|0.13385709732307427|
|  0| 0.5897562959687032|
|  0|0.01540012100242305|
|  0|0.22569943461197162|
|  0| 0.9207602095112212|
+---+-------------------+



# let us see , how many recordsa  are present 

In [33]:

df.count()

999999

# let us check the shuffle reads and writes

In [34]:
## to check number of paritions , we must first convert df to rdd
df.rdd.getNumPartitions()

6

In [35]:
## it will be much faster to access the data , if we use cached

df.cache().count()

999999

# when we pull back data to the driver ( e.g collect() , toPandas() , etc ),  you ll need  to be careful 
# of how much data  you are bringing back . Otherwise ,  you might get  OOM exceptional 
#  best practise ,  is explicitly ,  limit the number of records , unless your data set is  small ,
# before calling collect() ,  topandas()

In [36]:
df.limit(50).toPandas()

,id,v
0,0,0.133857
1,0,0.589756
2,0,0.015400
3,0,0.225699
4,0,0.920760
5,0,0.622282
6,0,0.102984
7,0,0.667876
8,0,0.067482
9,0,0.521518
